# Provided code

You shouldn't need to change anything in this section.

### Load data to Colab

In [2]:
if False:  # manual loading
    from google.colab import file
    uploaded = files.upload()  # then browse, select the files
    
else:  # automatic loading
    import requests
    import gzip
    
    filepath_d_gr = 'http://users.diag.uniroma1.it/challenge9/data/USA-road-d/' + 'USA-road-d.NY.gr.gz'
    filepath_t_gr = 'http://users.diag.uniroma1.it/challenge9/data/USA-road-t/' + 'USA-road-t.NY.gr.gz'
    filepath_d_co = 'http://users.diag.uniroma1.it/challenge9/data/USA-road-d/' + 'USA-road-d.NY.co.gz'
    
    def loader(url):
        name = url.rsplit('/', 1)[1].rsplit('.', 1)[0]
        savename = name + '.txt'
        
        with open(savename, 'wb') as f_out:
            with requests.get(url) as r:
                f_in = gzip.decompress(r.content)
                f_out.write(f_in)
                
        print(savename)
            
    loader(filepath_d_gr)
    loader(filepath_t_gr)
    loader(filepath_d_co)
    
    

USA-road-d.NY.gr.txt
USA-road-t.NY.gr.txt
USA-road-d.NY.co.txt


### Graph and Vertex classes

In [3]:
# Vertex implementation
class Vertex:
    # Initialization of a vertex, given a neighbor and the corresponding weight
    # Each vertex contains a list of neighbors and corresponding weights
    def __init__(self, i, neighbor_index, weight):
        self.index = i
        self.neighbors = [neighbor_index]
        self.weights = [weight]
        
    def getNeighbors(self):
        return self.neighbors
    
    def getWeights(self):
        return self.weights
    
    # Add a neighbor with corresponding weight to the vertex
    def _addNeighbor(self, neighbor_index, weight):
        self.neighbors.append(neighbor_index)
        self.weights.append(weight)


# Graph data structure
class Graph:
    # Initializes a graph with n_vertices nodes
    # The graph contains a list of vertices
    def __init__(self, n_vertices):
        self.vertices = [None] * (n_vertices+1)
        self.num_vertices = len(self.vertices)
        
    def getAmountVertex(self):
        return(self.num_vertices)
    
    # Returns the i'th node
    def getVertex(self, i):
        if ((i > len(self.vertices)) | (i <= 0)):
            raise ValueError(f'index {i} is out of bounds')
        else:
            return self.vertices[i]
    
    # Adds a new vertex to the graph
    def _addVertex(self, vertex_index, neighor_index, distance):
        if (self.vertices[vertex_index] == None):
            # Construct new vertex
            self.vertices[vertex_index] = Vertex(vertex_index, neighor_index, distance)
        else:
            # Vertex already in graph but other neighbor, add extra edge
            self.vertices[vertex_index]._addNeighbor(neighor_index, distance)




In [4]:
import fileinput

# Read graph data
def readGraph(filePath):
    n_vertices = 0
    for line in fileinput.input([filePath]):
        words = line.split(" ")
        if (words[0] == "p"):
            n_vertices = int(words[2])
    graph = Graph(n_vertices)
    for line in fileinput.input([filePath]):
        words = line.split(" ")
        if (words[0] == "a"):
            graph._addVertex(int(words[1]), int(words[2]), float(words[3]))
    return graph


# Read coordinates data
def readCoordinates(filepath):
    # Start to count from 1
    coordinates = [None]
    for line in fileinput.input([filepath]):
        words = line.split(" ")
        if (words[0] == "v"):
            coordinates.append([float(words[2]), float(words[3])])
    return coordinates


### Usefull functions

In [5]:
import numpy as np
    
# Priority queue definition
class PriorityQueue(dict):
    def put(self, item, value):
        # Watch out that value is not overwritten with higher value, shouldn't be allowed to happen!
        self[item] = value  
    
    def pop(self):
        """
        Returns the item with the lowest weight
        """
        item_min = min(self, key=self.get)
        super().pop(item_min)
        return item_min

    
def angles2centimeters(lo, la):
    """
    Convert longitude and latitude to local orthogonal grid
    :param lo: longitude
    :param la: latitude
    :return: height and width coordinates in cm's
    """
    
    radius = 6300 * 1e4  # cm
    la_mean = 40794234.  # 1e-6 degree
    lo_mean = -74016939.  # 1e-6 degree
    
    w = radius * np.cos(np.radians(la / 1e6)) * np.radians((lo - lo_mean) / 1e6)
    h = radius * np.radians((la - la_mean) / 1e6)
    
    return w, h 

# Assignment

First we'll check the amount of verteces and edges

## Code skeletons

Feel free to move the following code to the relevant questions. 

Before submitting your code, make sure to execute all code fields sequentially. Notebooks that don't execute sequentially will be penalised.

In [6]:

graph = readGraph('USA-road-d.NY.gr.txt')
co = readCoordinates('USA-road-d.NY.co.txt')

verteces_amount = graph.getAmountVertex()
print("sum of verteces", verteces_amount)

vertex_1 = graph.getVertex(1)

sum_of_edges = 0
for i in range(1, verteces_amount):
    tempVertex = graph.getVertex(i)
    neighbours_amount = len(tempVertex.getNeighbors())
    sum_of_edges += neighbours_amount
print("sum of edges", sum_of_edges)


sum of verteces 264347
sum of edges 733846


In [7]:
import sys
import math

max = sys.maxsize


# Heuristic function
def h(node1, node2):
    """
    Heuristic function
    """
    node1_co = co[node1]
    node2_co = co[node2]
    
    n1_h, n1_w = angles2centimeters(co[node1][0], co[node1][1])
    n2_h, n2_w = angles2centimeters(co[node2][0], co[node2][1])
    
    
    distance = math.sqrt(((n1_h - n2_h) ** 2) + ((n1_w - n2_w) ** 2))
    
    return distance

def get_distance(vertex, neighbor):
    current_vertex = graph.getVertex(vertex)
    weights = current_vertex.getWeights()
    neighbors = current_vertex.getNeighbors()
    return weights[neighbors.index(neighbor)]

# Algorithm
def a_star_search(graph, co, start, goal):
    """
    A* algorithm
    :param graph: Graph object
    :param co: coordinates list
    :param start: index of start node
    :param goal: index of goal node
    :return: The path of nodes and total length
    """
    list_closed = []
    list_open = []
    list_open.append(start)
    list_cameFrom = [None] * (verteces_amount)
    total_length = 0
    print(verteces_amount)
    print("len")
    print(len(list_cameFrom))
    
    f_dict = {i:max for i in range(1, verteces_amount)}
    g_dict = {i:max for i in range(1, verteces_amount)}
    f = PriorityQueue(f_dict)
    g = PriorityQueue(g_dict)
    
    g.put(start, 0)
    f.put(start, g[start] + h(start,goal))
      
    while len(list_open) > 0:
        current = f.pop()
        if current == goal:
            break
        list_open.remove(current)
        list_closed.append(current)
        for i in graph.getVertex(current).getNeighbors():
            if i in list_closed:
                continue
            tentative_g_score = g[current] + get_distance(current, i)
            if i not in list_open:
                list_open.append(i)
                #print(list_open)
            elif tentative_g_score >= g[i]:
                continue
            list_cameFrom[i] = current
            g[i] = tentative_g_score
            f[i] = g[i] + h(i, goal)
            
    #Iterate back from goal vertex to start vertex and calculating the distance
    previous_node = goal
    path = []
    total_length = 0
    while previous_node != start:
        temp = list_cameFrom[previous_node]
        path.append(temp)
        total_length += get_distance(previous_node,temp)
        previous_node = temp
    
    return len(path), total_length


##testing
a_star_search(graph, co, 48027, 18582)

264347
len
264347


(211, 243954.0)

In [15]:
##testing cell


import sys
max = sys.maxsize

#make sure to start jupyter-lab with the following flag: jupyter-lab --NotebookApp.iopub_data_rate_limit=1.0e10
#f_dict = {i:max for i in range(1, verteces_amount)}
#f = PriorityQueue(f_dict)
#print(f.pop())
print(co[1][0])


-73530767.0


In [6]:
# Check your solution with distances.txt. If A* is implemented correctly,
# (using Eucledian distance), it should return the same values for each  
# group number. Edit only the variables group_number and num_vertices.

import random

group_number = 0  # TODO: change to your group number
num_vertices = 0  # TODO: number of vertices in the graph

random.seed(group_number)

start = random.randint(1, num_vertices + 1)
goal = random.randint(1, num_vertices + 1)


# Calculating the path between nodes
path, path_length = a_star_search(graph, co, start, goal)

## Answers

Answer the questions from the assignment and add appropriate code where relevant to the question.

In [7]:
# TODO